<a href="https://colab.research.google.com/github/Riya4797/Ml/blob/main/Fakenews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re# to search the words in doc
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
news=pd.read_csv('/content/drive/MyDrive/train.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
news.describe()

,id,label
count,20800.000000,20800.000000
mean,10399.500000,0.500625
std,6004.587135,0.500012
min,0.000000,0.000000
25%,5199.750000,0.000000
50%,10399.500000,1.000000
75%,15599.250000,1.000000
max,20799.000000,1.000000


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
news.shape

(20800, 5)

In [ ]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
 #missing value analysis
 news.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
#  replacing null values
news=news.fillna('')

In [ ]:
 #mergin author and text
 news['content']=news['author']+' '+news['title']
 print(news['content'])


0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [ ]:
#seprating data and label
x=news.drop('label',axis=1)
y=news['label']

In [ ]:
x,y

(          id                                              title  \
 0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
 1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
 2          2                  Why the Truth Might Get You Fired   
 3          3  15 Civilians Killed In Single US Airstrike Hav...   
 4          4  Iranian woman jailed for fictional unpublished...   
 ...      ...                                                ...   
 20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
 20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
 20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
 20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
 20799  20799                          What Keeps the F-35 Alive   
 
                                           author  \
 0                                  Darrell Lucus   
 1                                Daniel J. Flynn   
 2                     

Stemming:
means to find root word


In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stem_content=re.sub('[^a-zA-Z]',' ',content)#to take alphabets and eexcluding everything else
  stem_content=stem_content.lower()
  stem_content=stem_content.split()
  stem_content=[port_stem.stem(word) for word in stem_content if not word in  stopwords.words('english')]
  stem_content=' '.join(stem_content)
  return stem_content

In [ ]:
news['content']=news['content'].apply(stemming)

In [ ]:
news['content']

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object

In [ ]:
#seprating data and label
x=news['content'].values
y=news['label'].values

In [ ]:
print(x,y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv'] [1 0 1 ... 0 1 1]


In [ ]:
y

array([1, 0, 1, ..., 0, 1, 1])

In [ ]:
y.shape

(20800,)

In [ ]:
#text to feature vector
vectorizer=TfidfVectorizer ()
vectorizer.fit(x)
X=vectorizer.transform(x)



In [ ]:
X

<20800x17128 sparse matrix of type '<class 'numpy.float64'>'
	with 210687 stored elements in Compressed Sparse Row format>

In [ ]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, stratify=y, random_state=3)

In [ ]:
 model=LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

LogisticRegression()

In [ ]:
X_train_pred=model.predict(X_train)
train_data_acc=accuracy_score(X_train_pred,y_train)

In [ ]:
print(train_data_acc)

0.9861778846153846


In [ ]:
X_test_pred=model.predict(X_test)
test_data_acc=accuracy_score(X_test_pred,y_test)

In [ ]:
print('test acc:',test_data_acc)

test acc: 0.9764423076923077


In [ ]:
#making a predictive sys

In [ ]:
X_new=X_test[0]
pred=model.predict(X_new)

In [ ]:
pred

array([1])

In [ ]:
if pred[0] == 0:
  print("real news")
else:
  print('fake news')

fake news


In [ ]:
y_test[0]

1

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
clf1=GaussianNB()
clf2= MultinomialNB()
clf3= BernoulliNB()
clf1.fit(X_train.toarray(),y_train)
clf2.fit(X_train,y_train)
clf3.fit(X_train,y_train)



BernoulliNB()

In [ ]:
X_train_pred=clf1.predict(X_train.toarray())
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for GaussianNB ')
X_test_pred=clf1.predict(X_test.toarray()) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print(' GaussianNB test acc:',test_data_acc)

0.9164663461538461 is training data accuracy for GaussianNB 
 GaussianNB test acc: 0.7949519230769231


In [ ]:
X_train_pred=clf2.predict(X_train)
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for  MultinomialNB()')
X_test_pred=clf2.predict(X_test) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print('  MultinomialNB() test acc:',test_data_acc)

0.9763822115384615 is training data accuracy for  MultinomialNB()
  MultinomialNB() test acc: 0.9584134615384615


In [ ]:
X_train_pred=clf3.predict(X_train)
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for BernoulliNB ')
X_test_pred=clf3.predict(X_test) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print(' BernoulliNB test acc:',test_data_acc)

0.9909254807692308 is training data accuracy for BernoulliNB 
 BernoulliNB test acc: 0.9788461538461538


In [ ]:
X_new=X_test[0]
pred=clf3.predict(X_new)

In [ ]:
pred

array([1])

In [ ]:
y_test[0]

1

In [ ]:
from sklearn import tree
clf4 = tree.DecisionTreeClassifier()
clf4.fit(X_train.toarray(),y_train)


DecisionTreeClassifier()

In [ ]:
X_train_pred=clf4.predict(X_train.toarray())
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for  DecisionTree')
X_test_pred=clf4.predict(X_test.toarray()) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print('   DecisionTree test acc:',test_data_acc)

1.0 is training data accuracy for  DecisionTree
   DecisionTree test acc: 0.9932692307692308


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf5 = RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
clf5.fit(X_train,y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [ ]:
X_train_pred=clf5.predict(X_train.toarray())
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for RandomForest')
X_test_pred=clf5.predict(X_test.toarray()) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print(' RandomForest test acc:',test_data_acc)

0.7657451923076923 is training data accuracy for RandomForest
 RandomForest test acc: 0.7600961538461538


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf6=KNeighborsClassifier()


In [ ]:
clf6.fit(X_train,y_train)

KNeighborsClassifier()

In [ ]:
X_train_pred=clf6.predict(X_train.toarray())
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for KNeighbors')
X_test_pred=clf6.predict(X_test.toarray()) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print(' KNeighbors test acc:',test_data_acc)

0.5357572115384616 is training data accuracy for KNeighbors
 KNeighbors test acc: 0.5271634615384615


In [ ]:
from sklearn import svm
clf7 = svm.SVC()

In [ ]:
clf7.fit(X_train,y_train)

SVC()

In [ ]:
X_train_pred=clf7.predict(X_train.toarray())
train_data_acc=accuracy_score(X_train_pred,y_train)
print(train_data_acc, 'is training data accuracy for SVM')
X_test_pred=clf7.predict(X_test.toarray()) 
test_data_acc=accuracy_score(X_test_pred,y_test)
print(' SVM test acc:',test_data_acc)

0.9990384615384615 is training data accuracy for SVM
 SVM test acc: 0.9918269230769231


In [ ]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_score,recall_score,f1_score
y_train_pred=cross_val_predict(clf1,X_train.toarray(),y_train,cv=3)
y_train_pred

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)
print('thses are accuracy measurments  for clf1')
print('precision' ,precision_score(y_train,y_train_pred))
print('recall', recall_score(y_train,y_train_pred))
print('F-1 score',f1_score(y_train,y_train_pred))
     



thses are accuracy measurments  for clf1
precision 0.8826053639846744
recall 0.6913565426170468
F-1 score 0.7753618310333222


In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred=cross_val_predict(clf2,X_train.toarray(),y_train,cv=3)
y_train_pred

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)
print('thses are accuracy measurments  for clf2')
print('precision' ,precision_score(y_train,y_train_pred))
print('recall', recall_score(y_train,y_train_pred))
print('F-1 score',f1_score(y_train,y_train_pred))


thses are accuracy measurments  for clf2
precision 0.9933713376640594
recall 0.8995198079231692
F-1 score 0.9441189441189441


In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred=cross_val_predict(clf3,X_train.toarray(),y_train,cv=3)
y_train_pred

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)
print('thses are accuracy measurments  for clf3')
print('precision' ,precision_score(y_train,y_train_pred))
print('recall', recall_score(y_train,y_train_pred))
print('F-1 score',f1_score(y_train,y_train_pred))

thses are accuracy measurments  for clf3
precision 0.9797483523067705
recall 0.9815126050420168
F-1 score 0.9806296851574213


In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred=cross_val_predict(clf4,X_train.toarray(),y_train,cv=3)
y_train_pred

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)
print('thses are accuracy measurments  for clf4')
print('precision' ,precision_score(y_train,y_train_pred))
print('recall', recall_score(y_train,y_train_pred))
print('F-1 score',f1_score(y_train,y_train_pred))

thses are accuracy measurments  for clf4
precision 0.9921724470134875
recall 0.9890756302521009
F-1 score 0.990621618372009


In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred=cross_val_predict(clf5,X_train.toarray(),y_train,cv=3)
y_train_pred

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)
print('thses are accuracy measurments  for clf5')
print('precision' ,precision_score(y_train,y_train_pred))
print('recall', recall_score(y_train,y_train_pred))
print('F-1 score',f1_score(y_train,y_train_pred))

thses are accuracy measurments  for clf5
precision 0.8208940129449838
recall 0.9744297719087635
F-1 score 0.8910967175321111


In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred=cross_val_predict(clf6,X_train.toarray(),y_train,cv=3)
y_train_pred

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)
print('thses are accuracy measurments  for clf6')
print('precision' ,precision_score(y_train,y_train_pred))
print('recall', recall_score(y_train,y_train_pred))
print('F-1 score',f1_score(y_train,y_train_pred))

thses are accuracy measurments  for clf6
precision 0.5092966360856269
recall 0.9996398559423769
F-1 score 0.6747974068071313


In [ ]:
from sklearn.model_selection import cross_val_predict
y_train_pred=cross_val_predict(clf7,X_train.toarray(),y_train,cv=3)
y_train_pred

from sklearn.metrics import confusion_matrix
confusion_matrix(y_train,y_train_pred)
print('thses are accuracy measurments  for clf7')
print('precision' ,precision_score(y_train,y_train_pred))
print('recall', recall_score(y_train,y_train_pred))
print('F-1 score',f1_score(y_train,y_train_pred))